# Import

In [1]:
import pandas as pd
import ast
import numpy as np
import re

In [2]:
import spacy
# nlp = spacy.load("en_core_web_sm")
nlp = spacy.load("en_core_web_lg")

# Structured Summary Generation

In [3]:
dfs = []
for subset in ['reddit', 'nyt', 'stack']:
    temp_df = pd.read_csv(f"../data/{subset}/source_comments_arg_extracted.csv")
    dfs += [temp_df]
    
processed_df = pd.concat(dfs)

In [4]:
mask = pd.notnull(processed_df['arg_propositions'])
processed_df = processed_df[mask]

In [5]:
processed_df['arg_propositions'] = processed_df['arg_propositions'].apply(lambda x: x.strip("\n"))
processed_df['arg_propositions'] = processed_df['arg_propositions'].apply(lambda x: re.sub("Proposition:", "", x))
processed_df['arg_propositions'] = processed_df['arg_propositions'].apply(lambda x: x.strip("\n"))
processed_df['arg_propositions'] = processed_df['arg_propositions'].apply(lambda x: x.strip("`"))
# processed_df['arg_propositions'] = processed_df['arg_propositions'].apply(lambda x: x.strip("[`\n]+"))
processed_df['arg_propositions'] = processed_df['arg_propositions'].apply(lambda x: re.sub("^[\n ]*\[\]", "", x))
processed_df['arg_propositions'] = processed_df['arg_propositions'].apply(lambda x: re.sub("^json", "", x))

In [6]:
mask = processed_df['arg_propositions'].apply(lambda x: len(re.findall("^[\n ]*\[", x)) == 0)
processed_df = processed_df[~mask]

In [7]:
processed_df['arg_propositions'] = processed_df['arg_propositions'].apply(lambda x: ast.literal_eval(x.strip()))

In [8]:
processed_df = pd.json_normalize(processed_df.to_dict(orient='records'), 'arg_propositions', 
                  ['questionText', 'comments', 'summary', 'id', 'subset', 'my_category'])

In [9]:
processed_df

,claim,ground,warrant,backing,qualifier,rebuttal,questionText,comments,summary,id,subset,my_category
0,I would like to see any of the Legendary Beast...,[],NaN,NaN,NaN,NaN,What should I draw next? A FB user said they...,"I'd like to see any of the Legendary Beasts, R...",Commenters suggest ideas of the next Pokemon f...,0,reddit,1
1,Suicune is a good choice,[],NaN,NaN,NaN,NaN,What should I draw next? A FB user said they...,Ooo I'm kinda thinking of Suicune now. Good ch...,Commenters suggest ideas of the next Pokemon f...,0,reddit,1
2,I will consider Suicune for sure,[Suicune is a good choice],NaN,NaN,NaN,NaN,What should I draw next? A FB user said they...,Ooo I'm kinda thinking of Suicune now. Good ch...,Commenters suggest ideas of the next Pokemon f...,0,reddit,1
3,"If I do not choose Suicune now, I will definit...",[Suicune is a good choice],NaN,NaN,NaN,NaN,What should I draw next? A FB user said they...,Ooo I'm kinda thinking of Suicune now. Good ch...,Commenters suggest ideas of the next Pokemon f...,0,reddit,1
4,Pancham,[],NaN,NaN,NaN,NaN,What should I draw next? A FB user said they...,Pancham,Commenters suggest ideas of the next Pokemon f...,0,reddit,1
...,...,...,...,...,...,...,...,...,...,...,...,...
22586,"Please select connect under reset, reset after...",[this will solve issues],NaN,NaN,NaN,NaN,"""Could not find Cortex-M device in the JTAG ch...",Solved: In periperipheral settings under debu...,Commentators propose many different possible s...,249,stack,1
22587,"The device works after rebooting the PC, clear...",[The drivers are installed automatically when ...,NaN,NaN,NaN,NaN,"""Could not find Cortex-M device in the JTAG ch...",I faced the same issue and after breaking my h...,Commentators propose many different possible s...,249,stack,1
22588,A basic LED blink program can be used for testing,"[The device works after rebooting the PC, clea...",NaN,NaN,NaN,NaN,"""Could not find Cortex-M device in the JTAG ch...",I faced the same issue and after breaking my h...,Commentators propose many different possible s...,249,stack,1
22589,my JTAG debugger was functioning well on both ...,[],NaN,NaN,NaN,NaN,"""Could not find Cortex-M device in the JTAG ch...",I had the same issue: my JTAG debugger was fun...,Commentators propose many different possible s...,249,stack,1


In [10]:
def remove_duplicate_claim_reason(grp):
    all_reasons = list(np.hstack(grp['ground']))
    
    mask = grp['claim'].isin(all_reasons)
    mask &= grp['ground'].str.len() == 0
    
    return grp[~mask]
    
processed_df = processed_df.groupby(['subset', 'id', 'questionText']).apply(remove_duplicate_claim_reason).reset_index(drop=True)

/tmp/ipykernel_1206/3817567751.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  processed_df = processed_df.groupby(['subset', 'id', 'questionText']).apply(remove_duplicate_claim_reason).reset_index(drop=True)


In [11]:
def form_other_claim(grp):
    all_claim = grp['claim'].tolist()
    
    grp['other_claim'] = grp['claim'].apply(lambda x: [c for c in all_claim if c != x])
    
    return grp

processed_df = processed_df.groupby(['subset', 'id', 'questionText']).apply(form_other_claim).reset_index(drop=True)

/tmp/ipykernel_1206/731753434.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  processed_df = processed_df.groupby(['subset', 'id', 'questionText']).apply(form_other_claim).reset_index(drop=True)


In [12]:
claim_cluster_df = pd.read_pickle("../data/stage_1_refined_new_llm_based_clustered_test_set_s1_entail_ratio_3_3.5_0.1.pkl")
claim_cluster_df['cluster_id'] = claim_cluster_df['clusters'].apply(lambda x: [i for i in range(len(x))])
temp_df = claim_cluster_df.explode(['cluster_id', 'clusters']).explode(['clusters'])
temp_df = temp_df.rename(columns={'clusters': 'claim'})
temp_2_df = processed_df[['subset', 'my_category', 'id', 'questionText', 'claim', 'ground']]
temp_2_df = temp_2_df.drop_duplicates(subset=['subset', 'my_category', 'id', 'questionText', 'claim'])
temp_df = temp_df.merge(temp_2_df)
claim_cluster_df = temp_df.groupby(['subset', 'my_category', 'id', 'questionText', 'cluster_id'], sort=False).agg({
    'claim': (lambda x: x.tolist()),
    'ground': (lambda x: x.tolist())
}).reset_index()
claim_cluster_df = claim_cluster_df.rename(columns={'claim': 'claim_cluster'})
claim_cluster_df

,subset,my_category,id,questionText,cluster_id,claim_cluster,ground
0,reddit,1,0,What should I draw next? A FB user said they...,0,[I would like to see any of the Legendary Beas...,"[[], [Suicune is a good choice], [Suicune is a..."
1,reddit,1,0,What should I draw next? A FB user said they...,1,[I would love to see Reshiram],[[If you have the time]]
2,reddit,1,0,What should I draw next? A FB user said they...,2,[Pancham],[[]]
3,reddit,1,0,What should I draw next? A FB user said they...,3,[Seel?],[[]]
4,reddit,1,0,What should I draw next? A FB user said they...,4,[You should girafarig],[[girafarig is the best pokemon ever created n...
...,...,...,...,...,...,...,...
4832,stack,1,249,"""Could not find Cortex-M device in the JTAG ch...",7,[my JTAG debugger was functioning well on both...,[[]]
4833,stack,1,249,"""Could not find Cortex-M device in the JTAG ch...",8,[the solution proposed by splatapus on the TI ...,"[[I had the same issue, I tried various propos..."
4834,stack,1,249,"""Could not find Cortex-M device in the JTAG ch...",9,[A basic LED blink program can be used for tes...,"[[The device works after rebooting the PC, cle..."
4835,stack,1,249,"""Could not find Cortex-M device in the JTAG ch...",10,[Check the probe settings],[[]]


In [13]:
# ground_cluster_df = pd.read_pickle("./stage_2_refined_new_llm_based_clustered_test_set_s1_3.5_3.0.pkl")
# ground_cluster_df = pd.read_pickle("./stage_2_refined_new_llm_based_clustered_test_set_s1_3.5_3.5.pkl")  # NOT YET TRIED
ground_cluster_df = pd.read_pickle("../data/stage_2_refined_new_llm_based_clustered_test_set_s1_3.5_2.5.pkl")
# ground_cluster_df = pd.read_pickle("./stage_2_refined_new_llm_based_clustered_test_set_s1_3.5_2.pkl")

# ground_cluster_df = pd.read_pickle("./stage_2_refined_new_llm_based_clustered_test_set_s1_3.0_3.0.pkl")
ground_cluster_df = ground_cluster_df.rename(columns={'clusters': 'ground_clusters'})
ground_cluster_df

,subset,id,questionText,cluster_id,ground_clusters
0,reddit,0,What should I draw next? A FB user said they...,0,[[Suicune is a good choice]]
1,reddit,0,What should I draw next? A FB user said they...,1,[[If you have the time]]
2,reddit,0,What should I draw next? A FB user said they...,4,[[girafarig is the best pokemon ever created n...
3,reddit,0,What should I draw next? A FB user said they...,5,[[I admire your steadfast courage in how aweso...
4,reddit,1,"Historical Hypocrisy (First, a disclaimer: T...",1,[[none of the women in the paintings from cent...
...,...,...,...,...,...
23262,stack,249,"""Could not find Cortex-M device in the JTAG ch...",1,[[this will solve issues]]
23263,stack,249,"""Could not find Cortex-M device in the JTAG ch...",3,"[[Only JTAG is supported], [The ICDI can be us..."
23273,stack,249,"""Could not find Cortex-M device in the JTAG ch...",5,[[Disabling all the drivers for the device in ...
23276,stack,249,"""Could not find Cortex-M device in the JTAG ch...",8,"[[I tried various proposed solutions, I had th..."


In [14]:
final_df = claim_cluster_df.merge(ground_cluster_df, how='left')
final_df

,subset,my_category,id,questionText,cluster_id,claim_cluster,ground,ground_clusters
0,reddit,1,0,What should I draw next? A FB user said they...,0,[I would like to see any of the Legendary Beas...,"[[], [Suicune is a good choice], [Suicune is a...",[[Suicune is a good choice]]
1,reddit,1,0,What should I draw next? A FB user said they...,1,[I would love to see Reshiram],[[If you have the time]],[[If you have the time]]
2,reddit,1,0,What should I draw next? A FB user said they...,2,[Pancham],[[]],NaN
3,reddit,1,0,What should I draw next? A FB user said they...,3,[Seel?],[[]],NaN
4,reddit,1,0,What should I draw next? A FB user said they...,4,[You should girafarig],[[girafarig is the best pokemon ever created n...,[[girafarig is the best pokemon ever created n...
...,...,...,...,...,...,...,...,...
4832,stack,1,249,"""Could not find Cortex-M device in the JTAG ch...",7,[my JTAG debugger was functioning well on both...,[[]],NaN
4833,stack,1,249,"""Could not find Cortex-M device in the JTAG ch...",8,[the solution proposed by splatapus on the TI ...,"[[I had the same issue, I tried various propos...","[[I tried various proposed solutions, I had th..."
4834,stack,1,249,"""Could not find Cortex-M device in the JTAG ch...",9,[A basic LED blink program can be used for tes...,"[[The device works after rebooting the PC, cle...","[[The device works after rebooting the PC, cle..."
4835,stack,1,249,"""Could not find Cortex-M device in the JTAG ch...",10,[Check the probe settings],[[]],NaN


In [15]:
mask = pd.isnull(final_df['ground_clusters'])
final_df[mask]

,subset,my_category,id,questionText,cluster_id,claim_cluster,ground,ground_clusters
2,reddit,1,0,What should I draw next? A FB user said they...,2,[Pancham],[[]],NaN
3,reddit,1,0,What should I draw next? A FB user said they...,3,[Seel?],[[]],NaN
6,reddit,1,1,"Historical Hypocrisy (First, a disclaimer: T...",0,[I can't repost this enough],[[]],NaN
8,reddit,1,1,"Historical Hypocrisy (First, a disclaimer: T...",2,"[Postimg is horrible, Your view on the image h...","[[], []]",NaN
11,reddit,1,1,"Historical Hypocrisy (First, a disclaimer: T...",5,[Hugely obese people back then were very rare ...,[[]],NaN
...,...,...,...,...,...,...,...,...
4829,stack,1,249,"""Could not find Cortex-M device in the JTAG ch...",4,[The Keil forums might have some more ideas yo...,[[]],NaN
4831,stack,1,249,"""Could not find Cortex-M device in the JTAG ch...",6,[Try lowering the JTAG frequency],[[]],NaN
4832,stack,1,249,"""Could not find Cortex-M device in the JTAG ch...",7,[my JTAG debugger was functioning well on both...,[[]],NaN
4835,stack,1,249,"""Could not find Cortex-M device in the JTAG ch...",10,[Check the probe settings],[[]],NaN


In [16]:
from collections import OrderedDict
final_df.loc[mask, 'ground_clusters'] = final_df.loc[mask, 'ground'].apply(lambda x: [list(OrderedDict.fromkeys(np.hstack(x)))])

In [17]:
final_df

,subset,my_category,id,questionText,cluster_id,claim_cluster,ground,ground_clusters
0,reddit,1,0,What should I draw next? A FB user said they...,0,[I would like to see any of the Legendary Beas...,"[[], [Suicune is a good choice], [Suicune is a...",[[Suicune is a good choice]]
1,reddit,1,0,What should I draw next? A FB user said they...,1,[I would love to see Reshiram],[[If you have the time]],[[If you have the time]]
2,reddit,1,0,What should I draw next? A FB user said they...,2,[Pancham],[[]],[[]]
3,reddit,1,0,What should I draw next? A FB user said they...,3,[Seel?],[[]],[[]]
4,reddit,1,0,What should I draw next? A FB user said they...,4,[You should girafarig],[[girafarig is the best pokemon ever created n...,[[girafarig is the best pokemon ever created n...
...,...,...,...,...,...,...,...,...
4832,stack,1,249,"""Could not find Cortex-M device in the JTAG ch...",7,[my JTAG debugger was functioning well on both...,[[]],[[]]
4833,stack,1,249,"""Could not find Cortex-M device in the JTAG ch...",8,[the solution proposed by splatapus on the TI ...,"[[I had the same issue, I tried various propos...","[[I tried various proposed solutions, I had th..."
4834,stack,1,249,"""Could not find Cortex-M device in the JTAG ch...",9,[A basic LED blink program can be used for tes...,"[[The device works after rebooting the PC, cle...","[[The device works after rebooting the PC, cle..."
4835,stack,1,249,"""Could not find Cortex-M device in the JTAG ch...",10,[Check the probe settings],[[]],[[]]


In [18]:
final_df = final_df.rename(columns={'questionText': 'query'})
final_df['arg_input'] = final_df.apply(lambda row: {'claim': row['claim_cluster'],
                                                    'ground': row['ground_clusters']}, axis=1)

In [19]:
final_df

,subset,my_category,id,query,cluster_id,claim_cluster,ground,ground_clusters,arg_input
0,reddit,1,0,What should I draw next? A FB user said they...,0,[I would like to see any of the Legendary Beas...,"[[], [Suicune is a good choice], [Suicune is a...",[[Suicune is a good choice]],{'claim': ['I would like to see any of the Leg...
1,reddit,1,0,What should I draw next? A FB user said they...,1,[I would love to see Reshiram],[[If you have the time]],[[If you have the time]],"{'claim': ['I would love to see Reshiram'], 'g..."
2,reddit,1,0,What should I draw next? A FB user said they...,2,[Pancham],[[]],[[]],"{'claim': ['Pancham'], 'ground': [[]]}"
3,reddit,1,0,What should I draw next? A FB user said they...,3,[Seel?],[[]],[[]],"{'claim': ['Seel?'], 'ground': [[]]}"
4,reddit,1,0,What should I draw next? A FB user said they...,4,[You should girafarig],[[girafarig is the best pokemon ever created n...,[[girafarig is the best pokemon ever created n...,"{'claim': ['You should girafarig'], 'ground': ..."
...,...,...,...,...,...,...,...,...,...
4832,stack,1,249,"""Could not find Cortex-M device in the JTAG ch...",7,[my JTAG debugger was functioning well on both...,[[]],[[]],{'claim': ['my JTAG debugger was functioning w...
4833,stack,1,249,"""Could not find Cortex-M device in the JTAG ch...",8,[the solution proposed by splatapus on the TI ...,"[[I had the same issue, I tried various propos...","[[I tried various proposed solutions, I had th...",{'claim': ['the solution proposed by splatapus...
4834,stack,1,249,"""Could not find Cortex-M device in the JTAG ch...",9,[A basic LED blink program can be used for tes...,"[[The device works after rebooting the PC, cle...","[[The device works after rebooting the PC, cle...",{'claim': ['A basic LED blink program can be u...
4835,stack,1,249,"""Could not find Cortex-M device in the JTAG ch...",10,[Check the probe settings],[[]],[[]],"{'claim': ['Check the probe settings'], 'groun..."


In [20]:
final_df

,subset,my_category,id,query,cluster_id,claim_cluster,ground,ground_clusters,arg_input
0,reddit,1,0,What should I draw next? A FB user said they...,0,[I would like to see any of the Legendary Beas...,"[[], [Suicune is a good choice], [Suicune is a...",[[Suicune is a good choice]],{'claim': ['I would like to see any of the Leg...
1,reddit,1,0,What should I draw next? A FB user said they...,1,[I would love to see Reshiram],[[If you have the time]],[[If you have the time]],"{'claim': ['I would love to see Reshiram'], 'g..."
2,reddit,1,0,What should I draw next? A FB user said they...,2,[Pancham],[[]],[[]],"{'claim': ['Pancham'], 'ground': [[]]}"
3,reddit,1,0,What should I draw next? A FB user said they...,3,[Seel?],[[]],[[]],"{'claim': ['Seel?'], 'ground': [[]]}"
4,reddit,1,0,What should I draw next? A FB user said they...,4,[You should girafarig],[[girafarig is the best pokemon ever created n...,[[girafarig is the best pokemon ever created n...,"{'claim': ['You should girafarig'], 'ground': ..."
...,...,...,...,...,...,...,...,...,...
4832,stack,1,249,"""Could not find Cortex-M device in the JTAG ch...",7,[my JTAG debugger was functioning well on both...,[[]],[[]],{'claim': ['my JTAG debugger was functioning w...
4833,stack,1,249,"""Could not find Cortex-M device in the JTAG ch...",8,[the solution proposed by splatapus on the TI ...,"[[I had the same issue, I tried various propos...","[[I tried various proposed solutions, I had th...",{'claim': ['the solution proposed by splatapus...
4834,stack,1,249,"""Could not find Cortex-M device in the JTAG ch...",9,[A basic LED blink program can be used for tes...,"[[The device works after rebooting the PC, cle...","[[The device works after rebooting the PC, cle...",{'claim': ['A basic LED blink program can be u...
4835,stack,1,249,"""Could not find Cortex-M device in the JTAG ch...",10,[Check the probe settings],[[]],[[]],"{'claim': ['Check the probe settings'], 'groun..."


In [21]:
import os
from openai import OpenAI
client = OpenAI(
    api_key = "<your-api-key-here"
)

In [22]:
base_prompt = """In this task you are presented with a community question, a JSON object containing lists of comments to support a claim and its ground.
The question asks the opinions of users, and can be generally answered by the list of comments under 'claim'. The 'claim' can then be further justified by lists (clusters) of comments under 'ground'

You were tasked to generate the general opinion implied by comments under 'claim' to answer the question, and also generate every ground supported by each list of comments under 'ground'

Perform the following actions to solve this task:
+ Generate a concise key point that captures salient opinions from the list of comments under 'claim' --> 'Key Point Claim'
+ For every list of comments under 'ground', generate a concise key point that captures salient opinions across the comment that support the 'Key Point Claim' --> 'Key Point Ground'

Note that the claim should be general (less than 10 tokens) and the ground should be specific and cover details as much as possible, and must have a sentiment

The output must be a JSON object following the below template:
{'claim': 'Key Point Claim', 'ground': ['Key Point Ground 1', 'Key Point Ground 2']}

Question: %s
Claim-Ground JSON input: %s

"""

In [24]:
print(base_prompt)

In this task you are presented with a community question, a JSON object containing lists of comments to support a claim and its ground.
The question asks the opinions of users, and can be generally answered by the list of comments under 'claim'. The 'claim' can then be further justified by lists (clusters) of comments under 'ground'

You were tasked to generate the general opinion implied by comments under 'claim' to answer the question, and also generate every ground supported by each list of comments under 'ground'

Perform the following actions to solve this task:
+ Generate a concise key point that captures salient opinions from the list of comments under 'claim' --> 'Key Point Claim'
+ For every list of comments under 'ground', generate a concise key point that captures salient opinions across the comment that support the 'Key Point Claim' --> 'Key Point Ground'

Note that the claim should be general (less than 10 tokens) and the ground should be specific and cover details as much

In [25]:
# row = final_df.iloc[0]
# row = final_df.iloc[1]
# row = final_df.iloc[10]
# row = final_df.iloc[15]
# row = final_df.iloc[2]
# row = final_df.iloc[20]
# row = final_df.iloc[10]
# row = final_df.iloc[15]
# row = final_df.iloc[15]

# row = final_df.iloc[25]
# row = final_df.iloc[26]
row = final_df.iloc[27]
# row = final_df.iloc[28]
# row = final_df.iloc[29]
# row = final_df.iloc[30]
# row = final_df.iloc[36]
# row = final_df.iloc[43]

# row = final_df.iloc[6]
# row = final_df.iloc[7]
# row = final_df.iloc[24]
# row = final_df.iloc[14]
# row = final_df.iloc[17]
###########
# mask = final_df['id'] == 45
# # row = final_df[mask].iloc[0]
# # row = final_df[mask].iloc[1]
# row = final_df[mask].iloc[2]
# # row = final_df[mask].iloc[3]
# # row = final_df[mask].iloc[4]
# # row = final_df[mask].iloc[5]
# # row = final_df[mask].iloc[6]
# # row = final_df[mask].iloc[9]

print(row['id'])
print(row['query'])
print(row['claim_cluster'])
print(len(row['ground_clusters']))
print(row['arg_input'])
prompt = base_prompt %(row['query'], row['arg_input'])

3
Bike racks - what to look for?   I'm in the market for a bike rack. I've been looking at the Saris Bones 2 and 3. The only difference I can tell between the them is that the 3 series has an extra arm for stability. Is the Bones 2 not to be trusted?      Any other brands/models I should be considering?
['If you cannot attach bikes to a roof rack or a hitch, use the bones', 'I would recommend getting a hitch installed', 'If you can attach bikes to a roof rack or a hitch, do that', "If you don't have a hitch, and have no interest of adding one, go with the Bones"]
1
{'claim': ['If you cannot attach bikes to a roof rack or a hitch, use the bones', 'I would recommend getting a hitch installed', 'If you can attach bikes to a roof rack or a hitch, do that', "If you don't have a hitch, and have no interest of adding one, go with the Bones"], 'ground': [['A hitch is not a bad investment', "even saris says 'if you can attach bikes to a roof rack or a hitch, do that'", 'Most of the time it can 

In [84]:
response = get_completion(prompt, model)

In [85]:
print(response)

{
  "claim": "Hitch or roof racks are preferred",
  "ground": [
    "Hitches are affordable, sturdy, and versatile, making them a worthwhile investment for better bike rack options."
  ]
}


In [134]:
print(response)

{
  "claim": "Hitch or roof racks are preferred",
  "ground": [
    "Hitches are affordable, sturdy, and offer future utility"
  ]
}


In [ ]:
def get_claim_ground_kp_completion(query, arg_input):
    prompt = base_prompt %(query, arg_input)

    retries = 5
    while retries > 0:
        try:
            response = get_completion(prompt, model)
            return response
        except Exception as e:
            if e:
                if "exceeded your current quota" in str(e).lower():
                    raise e
                print(e)
                print('Timeout error, retrying...')
                retries -= 1
                if "limit reached for" in str(e).lower():
                    time.sleep(30)
                else:
                    time.sleep(5)
            else:
                raise e

    print('API is not responding, moving on...')
    return None

In [ ]:
results = []
for i, row in final_df.iterrows():
    query = row['query']
    arg_input = row['arg_input']

    claim_ground_kp = get_claim_ground_kp_completion(query, arg_input)
    results += [claim_ground_kp]
    time.sleep(0.1)

In [ ]:
final_df['claim_ground_kps'] = results

In [17]:
for subset in ['reddit', 'nyt', 'stack']:
    temp_df = final_df[processed_final_df['subset'] == subset]
    temp_df.to_csv(f"../data/{subset}/summaries.csv", index=False)